# Deliverable 1

- Create a set of 2000 random coordinates
- Get the nearest city with `citypy`
- Perform an API call with OpenWeatherMap
- Retrieve the following information from the API call:
    - Latitude and longitude
    - Maximum temperature
    - Percent humidity
    - Percent cloudiness
    - Wind speed
    - Weather description (for example, clouds, fog, light rain, clear sky)
- Add the data to a new DataFrame.
- Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the Weather_Database folder.

## Generate Coordinates
---

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [62]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

print(f'Number of Coordinates: {len(coordinates)}\nSample:\n{coordinates[1:5]}')

Number of Coordinates: 2000
Sample:
[(-86.3104753757061, -176.6425028080113), (-86.91170115532368, -146.10346736192864), (18.240542039271077, 103.90771529140204), (-13.542508170993116, -56.37346483419694)]


## Map Nearest City
---

In [9]:
# Import the citypy module to determine nearest city based on coordinates
from citipy import citipy

In [66]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
print(f'Numer of Cities: {len(cities)}\nSample: {cities[1:5]}')

Numer of Cities: 770
Sample: ['vaini', 'mataura', 'pakxan', 'diamantino']


## API Call to OpenWeather
---

In [83]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [84]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [88]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


Reminder - Fields Needed:

- Latitude and longitude
- Maximum temperature
- Percent humidity
- Percent cloudiness
- Wind speed
- Weather description (for example, clouds, fog, light rain, clear sky)

In [89]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        # restart record count to 1
        record_count = 1
        # rest for 60 seconds
        time.sleep(60)

    # Create endpoint URL with each city - replaceing spaces in city name with '+'.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather["weather"][0]["description"]
        openWx_city_name = city_weather['name']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"CitiPy City": city.title(),
                          'OpenWx City': openWx_city_name,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          'Wx Description': city_weather_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print(f"City '{city}' not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | tamale
Processing Record 2 of Set 1 | vaini
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | pakxan
Processing Record 5 of Set 1 | diamantino
Processing Record 6 of Set 1 | provideniya
Processing Record 7 of Set 1 | faanui
Processing Record 8 of Set 1 | dudinka
Processing Record 9 of Set 1 | sao filipe
Processing Record 10 of Set 1 | dikson
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | aykhal
Processing Record 13 of Set 1 | tasiilaq
Processing Record 14 of Set 1 | east london
Processing Record 15 of Set 1 | twin falls
Processing Record 16 of Set 1 | grand gaube
Processing Record 17 of Set 1 | klaksvik
Processing Record 18 of Set 1 | kasongo-lunda
Processing Record 19 of Set 1 | kaitangata
Processing Record 20 of Set 1 | lebedyn
Processing Record 21 of Set 1 | esperance
Processing Record 22 of Set 1 | punta arenas
Processing Record 23 of Set 1 | hobart
Processing Record 24 of Set 1 | rikitea
Processing Recor

## Process JSON Output
---

In [90]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,CitiPy City,OpenWx City,Lat,Lng,Wx Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tamale,Tamale,9.4008,-0.8393,clear sky,82.54,13,1,6.89,GH,2022-01-23 20:11:31
1,Vaini,Vaini,-21.2000,-175.2000,clear sky,81.10,69,9,9.35,TO,2022-01-23 20:11:31
2,Mataura,Mataura,-46.1927,168.8643,overcast clouds,53.47,68,96,1.32,NZ,2022-01-23 20:11:31
3,Pakxan,Pakxan,18.3778,103.6601,clear sky,65.50,75,6,3.91,LA,2022-01-23 20:11:32
4,Diamantino,Diamantino,-14.4086,-56.4461,broken clouds,84.18,68,74,3.47,BR,2022-01-23 20:11:32
5,Provideniya,Provideniya,64.3833,-173.3000,light snow,6.08,97,100,7.52,RU,2022-01-23 20:11:32
6,Faanui,Faanui,-16.4833,-151.7500,overcast clouds,81.39,72,96,18.03,PF,2022-01-23 20:11:33
7,Dudinka,Dudinka,69.4058,86.1778,overcast clouds,-16.76,100,100,5.30,RU,2022-01-23 20:11:33
8,Sao Filipe,São Filipe,14.8961,-24.4956,overcast clouds,75.18,70,100,11.01,CV,2022-01-23 20:11:33
9,Dikson,Dikson,73.5069,80.5464,overcast clouds,-10.80,100,100,15.41,RU,2022-01-23 20:11:34


In [100]:
# Rename select columns to match desired output example
city_data_df = city_data_df.rename(columns={
    'OpenWx City': 'City',
    'Wx Description': 'Current Description'
    })
# Reorder and drop columns
city_data_reordered_df = city_data_df[['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description','Date']]
# Preview
city_data_reordered_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Tamale,GH,9.4008,-0.8393,82.54,13,1,6.89,clear sky,2022-01-23 20:11:31
1,Vaini,TO,-21.2000,-175.2000,81.10,69,9,9.35,clear sky,2022-01-23 20:11:31
2,Mataura,NZ,-46.1927,168.8643,53.47,68,96,1.32,overcast clouds,2022-01-23 20:11:31
3,Pakxan,LA,18.3778,103.6601,65.50,75,6,3.91,clear sky,2022-01-23 20:11:32
4,Diamantino,BR,-14.4086,-56.4461,84.18,68,74,3.47,broken clouds,2022-01-23 20:11:32


In [102]:
# Output to CSV with index renamed as 'City ID'
city_data_reordered_df.to_csv('Weather_Database/WeatherPy_Database.csv',index_label='City ID')